The Data sources are:

### 1. Liam's SPSS coded data
**File:** The Loop 2017 Final Interventions.xlsx

Exported as Excel from SPSS, keeping the variable names.

This file contains 1325 entries.


27 have null festival or sample numbers so can't be used, leaving 1298


One has sample number 12151, two have sample number 0 - these cannot be merged.


This leaves 1295 - all of which can be merged

### 2. Guy's cleaned up lab data
**File:** Loop 2017 Lab fixed data.xlsm

From: Dropbox/Testing/2017 results processing/Loop 2017 Lab fixed data.xlsm


Data is in the ‘Raw Lab Data’ sheet


This file contains 2544 entries


1900 entries start with F


621 entries begin with A (amnesty) so can't be merged


23 Begin with W? so can't be merged


Entry SGP2017 F0465 needs editing as 'Client gender' is FemaleaMalee 

### 3. Boomtown Intervention Questionnaire
**File:** BTReport 2017 - Form responses 3.csv

Exported from: https://docs.google.com/spreadsheets/d/15pdETY0HK-VbBcV-N0swt6ZrRBbeDnZR5RGDzfq95dg

This file contains 194 entries

### Merging the data

Merging the data on Festival and SampleNumber resulted in 1295 entries



In [1]:
# Module imports
import os
import numpy as np
import pandas as pd

In [64]:
spssdata = '/opt/random/The Loop 2017 Final Interventions.xlsx'

spss_df = pd.read_excel(spssdata)
print("Read in from SPSS ",len(spss_df))

# Change festival names
spss_df['Festival'].replace(['BoomTown', 'KC', 'SGP'], ['BT2017', 'KC2017', 'SGP2017'], inplace=True)

# Ensure all Sample numbers are consistent
# 1. Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
# There are 45 entries that go, but none of them have any valid data in
spss_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True)

# 2. Make all sample numbers a 4-digit code starting with F
spss_df['SampleNumber'] = spss_df['SampleNumber'].apply(lambda x: 'F{:04d}'.format(int(x)))

# Combine date and time columns into new single column
spss_df['Date'] = pd.to_datetime(spss_df['Date']) # Convert Date to datetime object
spss_df['Date & Time of intervention'] = spss_df.apply(lambda r : pd.datetime.combine(r['Date'], r['Time']), 1)

# Remove Day, Date, Time and SurveyID columns
spss_df.drop(['Day', 'Date', 'Time', 'SurveyID'], axis=1, inplace=True)

# Fix dodgy sample number
spss_df.at[spss_df['SampleNumber'] == 'F12151', 'SampleNumber'] = 'F1215'

print("Final SPSS ",len(spss_df)) # shows we are left with 1298 datasets

Read in from SPSS  1343
Final SPSS  1298


In [66]:
labdata = '/opt/random/Loop 2017 Lab fixed data.xlsm'
lab_df = pd.read_excel(labdata, sheet_name='Raw LabData')
print("Read in from LAB ",len(lab_df))

# Remame 'Event Name' and 'Sample Number' columns so they match
lab_df.rename(columns={'Event  Name': 'Festival', 'Sample Number': 'SampleNumber'}, inplace=True)

# Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
lab_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True) # This just drops one case

# Uppercase all sample numbers
labels = ['SampleNumber']
lab_df.loc[:, labels] = lab_df[labels].apply(lambda x: x.str.upper())

# Some sample numbers begin with W or F 
#print(len(lab_df[ ~ (lab_df['SampleNumber'].str.startswith('F') | lab_df['SampleNumber'].str.startswith('A')) ]))
print("Final LAB ",len(lab_df)) # shows we are left with 2543 datasets

Read in from LAB  2544
Final LAB  2543


In [67]:
dft = pd.merge(spss_df, lab_df, how='inner', on=['Festival','SampleNumber'])
print("%d entries were merged" % len(dft))

# For checking which entries can't be merged - check for right_only
#pd.merge(lab_df, spss_df, how='outer', indicator=True)

# Sort first by Festival, then SampleNumber
dft.sort_values(['Festival', 'SampleNumber'], ascending=True, inplace=True)

# Here we reorder columns that should be identical to:
# 1. spot data errors
# 2. remove duplicate columns once we're happy data is consistent
prefix_cols = ['Festival', 'SampleNumber',
             'Sample submission time', 'Date & Time of return', 'Date & Time of intervention', 
             'Client age', 'Age', 'Client gender', 'Gender', 'Bought as', 'SubmittedSubstanceAs']

# Get the list of columns excluding the ones in prefix_cols
cols = [c for c in dft.columns.tolist() if c not in prefix_cols]
# Prepend prefix_cols to create the new list
cols = prefix_cols + cols
# Reorder columns
dft = dft[cols]

# Uppercase all genders for consistency
labels = ['Client gender', 'Gender']
dft.loc[:, labels] = dft[labels].apply(lambda x: x.str.upper())
# Set any MISSING to be nan
dft.loc[:, labels] = dft.loc[:, labels].replace({'MISSING':np.nan})

# Dump to excel
writer = pd.ExcelWriter('merged.xlsx')
dft.to_excel(writer, 'MergedData', index=False)
writer.save()

1296 entries were merged


In [5]:
# # See which non-na ages don't match
# # 540 entries have valid ages
# print(len(dft))
# df = dft[pd.notnull(dft['Client age']) & pd.notnull(dft['Age'])]
# print(len(df))
# df = df[df['Client age'] != df['Age']]
# # 127 don't match
# print(len(df))
# # df.to_csv('foo.csv')


# Cross tab 'Client gender' and 'Gender
#print(dft['Client gender'].unique())
#print(dft['Gender'].unique())

# # Look where they don't match
# df = dft[pd.notnull(dft['Client gender']) & pd.notnull(dft['Gender'])]
# df = df[df['Client gender'] != df['Gender']]
# # 127 don't match
# print(len(df))
# df.to_csv('foo.csv')



### Strategy for merging Google Forms interventions with SPSS data

<pre>

Concerns
Cannot find samples for 2 F0000

polysubstance:    2 + illegal drigs and usual alcohol frequency

'Ethnictiy_other' : ???
'UnitsConsumed' : ???

# 'BeerCider' : 'How much beer have you had today?', ADD 'How much cider have you had today?',
# 'As_expected' 

  
Below are in Google Forms but not in SPSS
Volunteer Name
When was the last time you used this service?
What was your first sample number at this event? Did you take a photo or keep the ticket?
Have you had any other legal or illegal drugs today?
Have you ever taken any other drugs I didn't mention?
Which drugs have you used? [Non-prescribed opiods]
Are you planning to take any of these drugs later?


DISPOSAL PLAN
'I will ask the Loop to safely dispose of the rest of the sample in my possession'
'I will throw it away myself'
'I will give it away instead of taking it myself'
'I will take a smaller amount of it'
'I will take a larger amount of it'
'I will take the same amount as usual'

OTHER ACTIONS
'I will take it over a longer time period'
'I will be more careful about mixing it with other substances'
'I will keep it to take it elsewhere, after the festival'
'I will sell it'
'I will obtain more on site'
'I will warn my friends and acquaintances'
'I will warn others via social media and public websites'
'I will tell my dealer'
'I will return it to my dealer'
'I will return it to my dealer'


LIAM
FriendsPresent	Where there friends present when the sample was given in?
ConsumedAlcohol	Has the respondent consumed alcohol?
UnitsConsumed	How many units?
PrescribedDrugs	Are they on any prescribed medication?
OverTheCounter	Are they taking any over the counter medication?
ConcernsWithCurrentFeelings	Do they have any current concerns with how they're feeling at the moment?
WhatConcerns	If so, what concerns?
SubmittedSubstanceAs	What has the substance been submitted as?
Obtained	Where have the drugs been obtained?
EverHadSubstance	Have they ever taken the drug before?
WhereAndWhen	If so, when and where?
NegativeExperieces	Have you or anyone you know ever had any negative experiences taking this substance?
ConsumedFromBatchAlready	Have they consumed from this batch already?
PriorConcerns	Do you have any prior conerns about this particular sample from this particular batch?
Why	If yes, what concerns?
AccessedSupportBefore	Have they accessed support from a treatment service for your alcohol or drug use?
WantFurtherAdvice	After our conversation today, would you like to have any further advice or support from a treatment service for your alcohol or drug use?

liam_col_map = {
'a' :'I will ask the loop to safely dispose of the further substandes in my possession',
'b' :'I will throw them away',
'c' : 'I will take a smaller amount of it',
'd' : 'I will take a larger amount of it',
'e' : 'I will take the same amount as usual',
'f' : 'I will take it over a longer time period',
'g' : 'I will be more careful about mixing it with other substances',
'h' : 'I will give it away',
'i' : 'I will sell it',
'j' : 'I will obtain more on site',
'k' : 'I will warn my friends',
'l' : 'I will warn others via social media and public websites',
'm' : 'I will tell my dealer',
'n' : 'I will return it to my dealer',
'o' : 'I will ask for a refund from my dealer',
'p' : 'I will go to another dealer',
'q' : 'I will keep it to take it elsewhere after the festival',
'r' : 'I will do something else'
}

</pre>

In [167]:
#
# Attempt to disentangle the Form responses into a form they can be merged with Liam's data
#

PERIODS = ['ever', 'year', 'month', 'week', 'yesterday', 'today', 'tonight', 'today_tonight']

def includes_frequency(cell, period):
    "Return boolean indicating if this cell contains frequencies >= period"
    if isinstance(cell, float) and np.isnan(cell):
        return False # nan's is considered not having the value
    
    #periods = ['ever', 'year', 'month', 'week', 'yesterday', 'today', 'tonight']
    form_responses = ['Had in my life', 'Had in last year', 'Had in last month', 'Had in last week', 
                      'Had yesterday', 'Had today', '(Probably) planning later']
    
    assert period in PERIODS, "Invalid period: {0}".format(period)
    
    values = [v.strip() for v in cell.split(',')]   
    idx = PERIODS.index(period)
    # check if any of the periods >= this have been checked
    for i in range(idx, len(PERIODS) - 1):
        if form_responses[i] in values:
            return True
    return False

def get_value(column, period):
    "Return boolean Series indicating if this columns contains frequencies >= period"
    result = None
    if period == 'today_tonight':
        today = column.apply(includes_frequency, period='today')
        tonight = column.apply(includes_frequency, period='tonight')
        result = today | tonight
    else:
        result = column.apply(includes_frequency, period=period)
    return result

def add_columns(bt_df, label_map):
    """For all of the drugs in the label_map expand them out to match Liam's data and then delete the original column
    """
    for gdrug in label_map.keys():
        gcolumn_label = 'Which drugs have you used? [{}]'.format(gdrug)
        gcolumn = bt_df[gcolumn_label]
        sdrug = label_map[gdrug]
        # Add columns for each period
        for period in PERIODS:
            column_name = '{}_{}'.format(sdrug, period)
            bt_df[column_name] = get_value(gcolumn, period)

        # Delete the original column
        bt_df.drop([gcolumn_label], axis=1, inplace=True)

def freq_summary(drugs, column_prefix):
    """Run boolean OR (any) for the give set of drugs and put result in column with 
    name {period}_{column_prefix}"""
    for period in PERIODS:
        labels = ['{}_{}'.format(drug, period) for drug in drugs]
        clabel = '{}_{}'.format(column_prefix, period)
        bt_df[clabel] = bt_df[labels].any(axis=1)
        
bt_interventions = '/opt/random/BTReport 2017 - Form responses 3.csv'
date_cols = ['Timestamp']
bt_df = pd.read_csv(bt_interventions, engine="python", parse_dates=date_cols)

column_map_core = {'Cannabis' : 'cannabis',
                   'Cocaine' : 'cocaine',
                   'Ecstasy pills' : 'ecstasy',
                   'Nitrous (NOS, laughing gas)' : 'nitrous_oxide',
                   'MDMA crystal/powder' : 'mdma',
                   'Ketamine' : 'ketamine',
                   'Magic mushrooms' : 'mushrooms',
                   'LSD' : 'lsd',
                   'Mephedrone (M-Cat)' : 'Mephedrone',
                   'Synthetic cannabinoids ("Spice")' : 'spice_legals',
                   'A powder which I had no idea what it was' : 'unknown_powder',
                  }

# Add core drug columns
add_columns(bt_df, column_map_core)

# Extra columns that aren't present in Liam's data
column_map_extra = {'2C-B' : '2cb',
                    'Amphetamine (speed)' : 'speed',
                    'Codeine' : 'coedine',
                    'Valium or other benzodiazepines' : 'valium',
                   }
add_columns(bt_df, column_map_extra)

"""
Now need to add calculated data for the other drug uses:
Legal (any_legal):    balloons, poppers, spice, other legal highs
core drugs (core):    cannabis, cocaine, ecstasy, mdma, ketamine, mephodrone, speed, heroin
polydrug ():    2 + illegal drugs
polysubstance:    2 + illegal drigs and usual alcohol frequency
"""
        
all_drugs = list(column_map_core.values()) + list(column_map_extra.values())
# ['cannabis', 'cocaine', 'ecstasy', 'nitrous_oxide', 'mdma', 'ketamine', 'mushrooms', 'lsd', 'Mephedrone', 'spice_legals', 'unknown_powder', '2cb', 'speed', 'coedine', 'valium']
freq_summary(all_drugs, 'any')
        
legal_drugs = ['nitrous_oxide', 'spice_legals', 'coedine', 'valium']
freq_summary(all_drugs, 'any_legal')

core_drugs = ['cannabis', 'cocaine', 'ecstasy', 'mdma', 'ketamine', 'Mephedrone', 'speed']
freq_summary(core_drugs, 'core')

# Now poly drug use
illegal_drugs = ['cannabis', 'cocaine', 'ecstasy', 'mdma', 'ketamine', 'mushrooms', 'lsd', 'Mephedrone', '2cb', 'speed']
column_prefix = 'polydrug'
for period in PERIODS:
    labels = ['{}_{}'.format(drug, period) for drug in illegal_drugs]
    clabel = '{}_{}'.format(column_prefix, period)
    bt_df[clabel] = bt_df[labels].sum(axis=1) >= 2

# Now poly substance use - NEED TO DECIDE ON ALCOHOL COLUMN
# column_prefix = 'polysubstance'
# for period in PERIODS:
#     clabel = '{}_{}'.format(column_prefix, period)
#     labels = ['polydrug_{}'.format(period), 'polydrug_{}'.format(period)]
#     bt_df[clabel] = bt_df[labels].sum(axis=1) >= 2

# Delete the drug columns that don't match Liam's data
for drug in column_map_extra.values():
    labels = ["{}_{}".format(drug, period) for period in PERIODS]
    bt_df.drop(labels, axis=1, inplace=True)

# Covert all booleans to Yes/No strings
# 'polysubstance'
prefixes = list(column_map_core.values()) + ['any', 'any_legal', 'core', 'polydrug' ]
columns = []
for prefix in prefixes:
    for period in PERIODS:
        columns.append("{}_{}".format(prefix, period))
for column in columns:
    bt_df[column] = bt_df[column].map({True : 'yes', False : 'no'})
        
#bt_df.to_csv('foo.csv')
print("DONE1")

DONE1


In [168]:
# Now disentangle the disposals columns

# Map of Google Forms responses to Liam's categories
GMAP = {
    'I will ask the Loop to safely dispose of the rest of the sample in my possession' : 'a',
    'I will throw it away myself' : 'b',
    'I will take a smaller amount of it' : 'c',
    'I will take a larger amount of it' : 'd',
    'I will take the same amount as usual' : 'e',
    'I will take it over a longer time period' : 'f',
    'I will be more careful about mixing it with other substances' : 'g',
    'I will give it away instead of taking it myself' : 'h',
    'I will sell it' : 'i',
    'I will obtain more on site' : 'j',
    'I will warn my friends and acquaintances' : 'k',
    'I will warn others via social media and public websites' : 'l',
    'I will tell my dealer' : 'm',
    'I will return it to my dealer' : 'n',
    'I will ask for a refund from my dealer' : 'o',
    'I will go to another dealer' : 'p',
    'I will keep it to take it elsewhere, after the festival' : 'q',
    'I will do something else' : 'r'
}
CATEGORIES = list(sorted(GMAP.values()))

def get_disposal(cell):
    """Return a list of booleans depending on which categories were seen for this cell."""
    result = [False] * len(CATEGORIES)
    if isinstance(cell, float) and np.isnan(cell):
        return result # nan's is considered not having the value
    values = cell.split(',')
    for v in values:
        v = v.strip()
        if v in GMAP.keys():
            idx = CATEGORIES.index(GMAP[v])
            result[idx] = True
    return result

label1 = 'After hearing today’s test results and harm reduction advice from The Loop, what do you plan to do with the sample?'
label2 = 'What other actions will you do?'

# Create two Series with the results of parsing the two columns
series1 = bt_df[label1].apply(get_disposal).apply(pd.Series)
series2 = bt_df[label2].apply(get_disposal).apply(pd.Series)

# OR them to get the final column as a dataframe and name the columns accordingly
df_tmp = series1 | series2

# Rename columns to match
df_tmp.columns = CATEGORIES

# add to the bt_df
bt_df = pd.concat([bt_df, df_tmp], axis=1)

# CONVERT TRUE/FALSE TO Yes/No
for column in CATEGORIES:
    bt_df[column] = bt_df[column].map({True : 'yes', False : 'no'})

# Delete redundant columns
bt_df.drop([label1, label2], axis=1, inplace=True)

print("DONE")

DONE


In [169]:
# Now we fix all the remaining columns

# Map Columns in Google Forms to SPSS
d = {'Sample Number': 'SampleNumber',
 'Timestamp' : 'Date & Time of intervention',
 'Number of friends present with primary respondent': 'FriendsPresent',
 'Gender of primary respondent': 'Gender',
 'Ethnicity': 'Ethnicity',
 'Age': 'Age',
 'Have you had any alcohol to drink today?': 'ConsumedAlcohol',
 'How much spirits have you had today?': 'Spirits',
 'How much wine have you had today?': 'Wine',
 'How many alcopops have you had today?': 'Alcopops',
 'Are you currently taking any prescribed medication?': 'PrescribedDrugs',
 'Are you currently taking any "Over the Counter" medication?': 'OverTheCounter',
 'Do you have any concerns about how you are feeling at the moment?': 'ConcernsWithCurrentFeelings',
 'You submitted a substance of concern for analysis, what do you believe it to be?': 'SubmittedSubstanceAs',
 'Where did you obtain the sample?': 'Obtained',
 'Very roughly, how often do you use this drug?': 'EverHadSubstance',
 'When did you first use this batch?': 'WhereAndWhen',
 'Have you or anyone you know ever had negative experiences taking this substance?': 'NegativeExperieces',
 'How many times have you used this batch?': 'ConsumedFromBatchAlready',
 'Do you have any concerns about using this sample from this batch or any other concerns about the result?': 'PriorConcerns',
 'Have you ever accessed a treatment service for your alcohol or drug use?': 'AccessedSupportBefore',
 'After our conversation today, would you like to have any further advice or support from a treatment service for your alcohol or drug use?': 'WantFurtherAdvice'
}

bt_df.rename(columns=d, inplace=True)

# Add Festival Column
bt_df['Festival'] = 'BT2017'

# Delete all columns that are only in Google Forms
to_drop = ['Volunteer Name', 
    'When was the last time you used this service?',
    'What was your first sample number at this event? Did you take a photo or keep the ticket?',
    'Which drugs have you used? [Non-prescribed opiods]',
    'Have you had any other legal or illegal drugs today?',
    'Have you ever taken any other drugs I didn\'t mention?',
    'Are you planning to take any of these drugs later?']

bt_df.drop(to_drop, axis=1, inplace=True)

# 'BeerCider' : 'How much beer have you had today?', ADD 'How much cider have you had today?',
# 'As_expected' 

# ConcernsWithCurrentFeelings' and any non yes/no answers to'WhatConcerns'
label1 = 'ConcernsWithCurrentFeelings'
label2 = 'WhatConcerns'
bt_df[label1].fillna('No', inplace=True)
mask = bt_df[label1] == 'No'
bt_df[label2] = np.nan
bt_df[label2].where(mask, bt_df[label1], inplace=True) # Copy values from label1 column over
bt_df[label1].where(mask, 'Yes', inplace=True) # Set copied over values to 'Yes'

label1 = 'PriorConcerns'
label2 = 'Why'
bt_df[label1].fillna('No', inplace=True)
mask = bt_df[label1] == 'No'
bt_df[label2] = np.nan
bt_df[label2].where(mask, bt_df[label1], inplace=True) # Copy values from label1 column over
bt_df[label1].where(mask, 'Yes', inplace=True) # Set copied over values to 'Yes'

# PriorConcerns': and any non yes/no answers to 'Why'
print("DONE RENAME")
bt_df.to_csv('foo.csv')

DONE RENAME


In [170]:
# Finally we need to make the remining columns match
s = spss_df.columns.values
b = bt_df.columns.values
print(set(s).difference(set(b)))
print(set(b).difference(set(s)))

{'other_today_tonight', 'polysubstance_month', 'polysubstance_year', 'other_ever', 'Ethnictiy_other', 'polysubstance_week', 'other_tonight', 'other_month', 'As_expected', 'UnitsConsumed', 'polysubstance_tonight', 'polysubstance_today', 'Cannabis_ever', 'polysubstance_today_tonight', 'spice_yesterday', 'other_yesterday', 'polysubstance_ever', 'polysubstance_yesterday', 'other_year', 'other_week', 'other_specify', 'BeerCider', 'other_today'}
{'spice_legals_yesterday', 'How much cider have you had today?', 'How much beer have you had today?', 'cannabis_ever'}
